In [1]:
!pip install -qqq bitsandbytes==0.43.1 trl peft==0.10.0 accelerate==0.28.0 transformers==4.39.3 flash-attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 52.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.2 

In [2]:
import torch
from datasets import Dataset, load_dataset
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from trl import SFTTrainer
from sklearn.model_selection import train_test_split

In [3]:
system_prompt=""

In [4]:
DataSet = load_dataset("messawey/EG_Historical_QA")

train_data, test_data = train_test_split(DataSet['train'], test_size=0.2, random_state=2024)

Generating train split:   0%|          | 0/420 [00:00<?, ? examples/s]

In [5]:
train_data['MergedColumn'][0]

'<s>[INST] ما هي أهمية كنوز توت عنخ امون؟ [/INST] ترجع أهمية مجموعة الملك توت عنخ آمون إلى العديد من الأسباب؛ أولها أن كنز الملك توت عنخ أمون هو أكمل كنز ملكي عُثر عليه ولا نظير له، إذ يتكون من ثلاثمائة وثمان وخمسين قطعة تشمل القناع الذهبي الرائع وثلاثة توابيت على هيئة الإنسان، أحدها من الذهب الخالص والآخران من خشب مذهب. ثانياً: أن تلك الأمتعة ترجع إلى الأسرة الثامنة عشرة أشهر وأزهى عصور الدولة الحديثة حيث انفتحت البلاد على أقاليم الشرق الأدنى القديم بفضل الحملات العسكرية والعلاقات التجارية من تصدير واستيراد للموارد والمنتجات المصنعة ونشاط أهل الحرف والفنانين. وأخيراً: أن هذه المجموعة الهائلة قد ظلت في مصر، وتوضح كيف كان القبر الملكي يجهز'

In [6]:
test_data['MergedColumn'][0]

'<s>[INST]  ما هو تأثير محمد علي باشا على التركيبة السكانية لمصر؟ [/INST] كان لحكم محمد علي باشا آثار ديموغرافية، حيث حدثت تغيرات في التوزيع السكاني بسبب الإصلاحات الاقتصادية والزراعية.'

In [7]:
MODEL_NAME="unsloth/llama-3-8b-Instruct-bnb-4bit"

def create_model_and_tokenizer():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto",
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer

In [8]:
model, tokenizer = create_model_and_tokenizer()
model.config.use_cache = False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:159: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Per-Training Results

In [9]:
prompt ="متى حكمت حتشبسوت مصر؟"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] متى حكمت حتشبسوت مصر؟ [/INST] 2021-02-16T14:30:00Z
2021-02-16T14:30:00Z <s>[INST] متى حكمت حتشبسوت مصر؟ [/INST] 2021-02-16T14:30:00Z
2021-02-16T14:30:00Z <s>[INST] متى حكمت حتشبسوت مصر؟ [/INST] 2021-02-16T14:30:00Z
2021-02-16T14:30:00Z <s>[INST] متى حكمت حتشبسوت مصر؟ [/INST] 2021-02-16T14:30:00Z
2021-02-16T14:30:00Z <s>[INST] متى حكمت حتشبس


In [10]:
prompt = "ما هو الغرض من بناء هرم خوفو؟ "
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] ما هو الغرض من بناء هرم خوفو؟  [/INST]  What is the purpose of building the Khufu's Pyramid?  </s>

<s>[INST] ما هو الغرض من بناء هرم خوفو؟  [/INST]  What is the purpose of building the Khufu's Pyramid?  </s>

<s>[INST] ما هو الغرض من بناء هرم خوفو؟  [/INST]  What is the purpose of building the Khufu's Pyramid?  </s>

<s>[INST] ما هو الغرض من بناء هرم خوفو؟  [/INST]  What is the purpose of building the Khufu's Pyramid?  </s>

<s>[INST] ما هو الغرض من بناء هرم خوفو؟  [/INST]  What is the purpose of building the Khufu's Pyramid?  </s>

<s>[INST] ما


In [11]:
lora_r = 16
lora_alpha = 64
lora_dropout = 0.1

peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    bias="none",
    task_type="CAUSAL_LM",
    use_dora=True
)

In [12]:
output_dir = './results'

# based on config
training_arguments = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=10,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir=output_dir,
    report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
)

In [13]:
train_data = Dataset.from_dict(train_data)
test_data = Dataset.from_dict(test_data)

In [14]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    peft_config=peft_config,
    dataset_text_field="MergedColumn",
    max_seq_length=64,
    tokenizer=tokenizer,
    args=training_arguments,
)

# To clear out cache for unsuccessful run
torch.cuda.empty_cache()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1847: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override

Map:   0%|          | 0/336 [00:00<?, ? examples/s]

Map:   0%|          | 0/84 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [15]:
train_result = trainer.train()

Step,Training Loss,Validation Loss
84,1.245300,1.364875
168,0.670300,1.346749
252,0.482100,1.511911
336,0.279100,1.647206
420,0.217200,1.731397


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

## Fine Tuning Results

In [16]:
prompt ="متى حكمت حتشبسوت مصر؟"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] متى حكمت حتشبسوت مصر؟ [/INST] حكمت حتشبسوت مصر عام 1479 قبل الميلاد.
